# **Import libraries and modules**

## Import libraries

In [ ]:
import os
import sys
import pathlib
import shutil
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import seaborn as sn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import modules

In [ ]:
sys.path.insert(0, '../')

from generate_version import generate_version

# **Prepare the dataset for loading**

## Helper functions

In [ ]:
def create_label_dir(df, dir='../gdsc-ai-challenge/train'):
    """Use Dataframe contains labels for each image and path to the directory

    contains the unlabeled dataset to rebuild directory into labeled subdirectories.

    Returns all the label and number of classes in the dataset.

    Keyword arguments:

    df -- The Dataframe contains images' names and labels.

    dir -- Path to the main directory (default to ../gdsc-ai-challenge/train)
    """
    class_names = np.sort(df['label'].unique())
    number_of_classes = len(class_names)

    if not os.path.exists(dir):
        return class_names, number_of_classes

    for class_name in class_names:
        subdir = pathlib.Path(os.path.join(dir, class_name))
        if subdir.exists():
            continue
        else:
            subdir.mkdir()
    
    return class_names, number_of_classes

def sort_data(df, dir='../gdsc-ai-challenge/train'):
    """Use Dataframe to move each unlabeled image to the correct label's subdirectory.

    df -- The Dataframe contains images' names and labels.

    dir -- Path to the main directory (default to ../gdsc-ai-challenge/train) 
    """
    if not os.path.exists(dir):
        return
    
    unlabeled_dir = os.path.join(dir, 'train')

    for image_dir in [str(img) for img in list(pathlib.Path(unlabeled_dir).glob('*.png'))]:
        id = int(image_dir.removeprefix(unlabeled_dir).removesuffix('.png'))
        label = df['label'][id - 1]
        dest_path = os.path.join(dir, label, str(id) + '.png')
        shutil.move(image_dir, dest_path)

## Prepare the dataset

In [ ]:
label_df = pd.read_csv('../gdsc-ai-challenge/trainLabels.csv')

class_names, number_of_classes = create_label_dir(label_df)
sort_data(label_df)

# **Data preprocessing and augmentation**

## Helper functions

In [ ]:
def split_dataset(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1):
    """Split the dataset into three subsets: train, validation (dev) and test set.

    Returns three tuples, containing each subset with its size.

    Keyword arguments:

    ds -- tf.data.Dataset object

    ds_size -- size of the dataset

    train_split -- percentage to split into train set (default to 0.8)

    val_split -- percentage to split into validation set (default to 0.1)

    test_split -- percentage to split into test set (default to 0.1)
    """
    assert (train_split + test_split + val_split) == 1
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return (train_ds, train_size), (val_ds, val_size), (test_ds, ds_size - val_size - train_size)

def configure(ds, ds_size, batch_size=32, shuffle=True, augment=False):
    """Configure the given dataset for better performance (by caching, prefetching and then batching the dataset)

    and perform preprocessing to the images in the given dataset.

    Returns the optimized dataset.

    Keyword arguments:

    ds -- tf.data.Dataset object

    ds_size -- size of the dataset

    batch_size -- size of each batch (default to 32)

    shuffle -- whether to shuffle the dataset (default to True)

    augment -- whether to perform data augmentation to the dataset (default to False)
    """

    AUTOTUNE = tf.data.AUTOTUNE
    rescale = keras.layers.Rescaling(1.0/255)
    data_augmentation = keras.Sequential([
        keras.layers.RandomFlip('horizontal'),
        keras.layers.RandomRotation(0.05, fill_mode='nearest')
    ])

    ds = ds.map(lambda x, y: (rescale(x), y),
                num_parallel_calls=AUTOTUNE)

    ds = ds.cache()
    if shuffle:
        ds = ds.shuffle(buffer_size=int(ds_size * 0.6))
    
    ds = ds.batch(batch_size)

    if augment:
        with tf.device('/cpu:0'):
            #only perform data augmentation on train set
            ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                                    num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

## Load the dataset using *image_dataset_from_directory()*

In [ ]:
ds = tf.keras.utils.image_dataset_from_directory(
    '../gdsc-ai-challenge/train',
    color_mode='grayscale',
    batch_size=None,
    image_size=(32,32),
    seed=42
)

ds_size = ds.cardinality().numpy()

## Split the dataset and Preprocess the dataset

In [ ]:
(train_ds, train_size), (val_ds, val_size), (test_ds, test_size) = split_dataset(ds, ds_size, train_split=0.7, val_split=0.2, test_split=0.1)

train_ds = configure(train_ds, train_size, augment=True)
val_ds = configure(val_ds, val_size)
test_ds = configure(test_ds, test_size, shuffle=False)

# **Create version-controlled folder for weights file**

In [ ]:
version = input("""Create new folder?

                (Y/n)    
                """)

new_version, path, save_path = generate_version('../Model/aiseries', version.lower() == 'y')

# **Build a model**

In [ ]:
%% write_and_run {path}/model.py
from tensorflow import keras
from keras import regularizers
import os

def create_model(path_to_weights=None, load_weights=True):
    """Function to create a model

    Returns a compiled and optionally loaded model

    Keyword arguments:

    path_to_weights -- (Optional, only used when load_weights is True) -- Path to weight file (.hdf5 files)

    load_weights -- Whether to load weights or not (default to True)
    """

    assert(load_weights == True and 
           path_to_weights is not None and 
           os.path.isfile(path_to_weights)), "path_to_weights must exist and not be NoneType if load_weights is True, otherwise change load_weights to False"

    model = keras.models.Sequential([
        keras.layers.Input((32,32,1)),
        keras.layers.Conv2D(64, (3,3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-3),
                            activity_regularizer=regularizers.l2(1e-4),
                            kernel_initializer='he_normal',
                            activation='elu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, (3,3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-3),
                            activity_regularizer=regularizers.l2(1e-4),
                            kernel_initializer='he_normal',
                            activation='elu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=(2,2)),

        keras.layers.Conv2D(128, (3,3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-3),
                            activity_regularizer=regularizers.l2(1e-4),
                            kernel_initializer='he_normal',
                            activation='elu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(128, (3,3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-3),
                            activity_regularizer=regularizers.l2(1e-4),
                            kernel_initializer='he_normal',
                            activation='elu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=(2,2)),

        keras.layers.Conv2D(256, (1,1), padding='same',
                            kernel_regularizer=regularizers.l2(1e-3),
                            activity_regularizer=regularizers.l2(1e-4),
                            kernel_initializer='he_normal',
                            activation='elu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, (1,1), padding='same',
                            kernel_regularizer=regularizers.l2(1e-3),
                            activity_regularizer=regularizers.l2(1e-4),
                            kernel_initializer='he_normal',
                            activation='elu'),
        keras.layers.BatchNormalization(),
        keras.layers.AveragePooling2D(pool_size=(2,2)),

        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='elu',
                        kernel_regularizer=regularizers.l2(1e-3),
                        activity_regularizer=regularizers.l2(1e-4)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation='elu',
                        kernel_regularizer=regularizers.l2(1e-3),
                        activity_regularizer=regularizers.l2(1e-4)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])

    if load_weights:
        model.load_weights(path_to_weights)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0003),
                                                loss='sparse_categorical_crossentropy',
                                                metrics=['accuracy'])

    return model

In [ ]:
model = create_model(load_weights=False)

model.summary()

# **Training session**

## Design a callback to stop training

In [ ]:
class stopCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}) :
        if (logs.get('accuracy') >= 0.999 and 
            logs.get('val_accuracy') >= 0.999) :
            print('\nReached 99.9% accuracy so stopping training')
            self.model.stop_training = True

callback = stopCallback()

#callback to save weights with the minimum loss value
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_path,
                                                               monitor='val_loss',
                                                               mode='min',
                                                               save_best_only=True)

In [ ]:
history = model.fit(train_ds, 
                    epochs=50, 
                    callbacks=[callback, model_checkpoint_callback], 
                    validation_data=val_ds)

# **Model evaluation**

## Evaluate on the test set using the best model

In [ ]:
model.load_weights('../Model/aiseries/version1.hdf5')

model.evaluate(test_ds)

## Evaluate based on training's metrics history

### 1. Based on loss value

In [ ]:
number_of_epochs = len(history.history['loss'])

In [ ]:
plt.plot(history.history['loss'], color='red', label='Train loss')
plt.plot(history.history['val_loss'], color='blue', label='Validation loss')

plt.xticks(np.arange(number_of_epochs, step=4))
plt.ylim((0, 1))
plt.legend()
plt.show()

### 2. Based on accuracy

In [ ]:
plt.plot(history.history['accuracy'], color='red', label='Training accuracy')
plt.plot(history.history['val_accuracy'], color='blue', label='Validation accuracy')

plt.xticks(np.arange(number_of_epochs, step=4))
plt.legend()
plt.show()

## Evaluate with Confusion Matrix and Classification Report

### Generate actual and predicted value

In [ ]:
y_true = np.concatenate([y for _, y in test_ds], axis=0)

Y_pred = model.predict(test_ds)
y_pred = np.argmax(Y_pred, axis=1)

### Plot Confusion matrix

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                                        display_labels=[class_name.capitalize() for class_name in class_names],
                                        cmap='Blues')

plt.xticks(rotation=60)
plt.savefig('../TrainingReport/confusion_matrix.pdf')
plt.show()

### Plot classification report

In [ ]:
clf_rep = classification_report(y_true, y_pred, 
                                target_names=[class_name.capitalize() for class_name in class_names], 
                                output_dict=True)

sn.heatmap(pd.DataFrame(clf_rep).iloc[:-1,:].T, annot=True)
plt.savefig('../TrainingReport/report.pdf')